<a href="https://colab.research.google.com/github/Prof-Luis1986/2A_practica01/blob/main/Tutorial_Ollama_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **🧪 Práctica guiada: Chat con Flet + Ollama (local)**

Objetivo

Que el alumnado:

Instale y pruebe Ollama y un modelo local.

Cree un proyecto Flet con flet create.

Arme la UI básica, conecte con Ollama y muestre la respuesta en vivo (streaming).

Añada TTS al final de cada respuesta.

Ajuste tokens para rendimiento.

0) Requisitos

Python 3.10+

Internet (solo para descargar Ollama y el modelo la primera vez)

1) Instalar Ollama
Windows

Descargar e instalar Ollama desde su web (instalador .exe).

Abrir PowerShell y comprobar:

In [ ]:
ollama --version


Si no arranca solo, ejecuta:

In [ ]:
ollama serve


macOS

Instalar con el .pkg (o vía Homebrew: brew install ollama).

Verificar en Terminal:

In [ ]:
ollama --version


2) Descargar el modelo (ligero y rápido)

Usaremos Qwen 2.5 (3B) para baja latencia.

In [ ]:
ollama pull qwen2.5:3b


Comprueba lo instalado:

In [ ]:
ollama list


3) Crear el proyecto con Flet

Instalar Flet (si hace falta):

In [ ]:
pip install flet


Crear la proyecto:

In [ ]:
flet create Appollama_voz
cd Appollama_voz


(Recomendado) Entorno virtual:
Windows (PowerShell):

In [ ]:
python -m venv .venv
.\.venv\Scripts\activate


macOS:

In [ ]:
python -m venv .venv
source .venv/bin/activate


Dependencias extra:

In [ ]:
pip install requests pyttsx3


4) Probar Flet “Hola mundo”

Abre main.py y deja este mínimo:

In [ ]:
import flet as ft

def main(page: ft.Page):
    page.title = "Hola Flet"
    page.add(ft.Text("¡Hola, Flet!"))

ft.app(target=main)


Ejecuta.

# ***Estructuremos el  chat con ollama.***

Paso 1 — Imports

In [ ]:
import flet as ft
import requests
import json
import pyttsx3
import platform
import os

¿Qué hace este bloque?

* Carga las librerías que la app necesita:

* flet: para construir la interfaz gráfica.

* requests: para conectarnos con el modelo Ollama vía HTTP.

* json: para leer e interpretar las respuestas del modelo.

* pyttsx3: para convertir texto a voz (TTS).

* platform y os: para detectar el sistema operativo y ejecutar comandos en el sistema.

Errores típicos:

* ModuleNotFoundError: falta instalar las librerías con pip.

Buenas prácticas:

* Mantener todos los imports organizados al inicio del archivo.

Paso 2 — Detección de sistema y selección de personaje/voz

In [ ]:
# ======== CONFIG ========
SO = platform.system()

if SO == "Darwin":  # Mac
    PERSONAJE = "Albert Einstein"
    EMOJI_PERSONAJE = "🧑‍🔬"
    VOZ = "Juan"
elif SO == "Windows":
    PERSONAJE = "Marie Curie"
    EMOJI_PERSONAJE = "👩‍🔬"
    VOZ = "Sabina"  # o "Zira"
else:
    PERSONAJE = "Personaje"
    EMOJI_PERSONAJE = "🧑‍🔬"
    VOZ = None

¿Qué hace este bloque?

* Detecta el sistema operativo en el que corre la app.

* Según el sistema, define qué personaje histórico aparecerá, con su emoji y voz.

* En otros sistemas (Linux, etc.) desactiva la voz.

Errores típicos:

* Que la voz seleccionada no exista en tu sistema.

Buenas prácticas:

* Ajustar los nombres de voz según el sistema operativo usado.

Paso 3 — Emojis, endpoint de Ollama y modelo

In [ ]:
EMOJI_USUARIO = "🧑‍💻"
OLLAMA_URL = "http://localhost:11434/api/generate"

# Modelo ligero para velocidad
MODEL = "qwen2.5:3b"  # puedes probar "gemma3:latest" o "mistral:7b"

# Opciones ajustadas para rapidez
OLLAMA_OPTIONS = {
    "num_ctx": 4096,
    "num_predict": 512,
    "temperature": 0.7,
    "top_p": 0.9,
    "repeat_penalty": 1.1,
}
KEEP_ALIVE = "30m"

¿Qué hace este bloque?

* Define el emoji del usuario.

* Indica la dirección del servidor local de Ollama.

* Selecciona el modelo de IA a utilizar.

* Configura parámetros de la generación de texto.

* Mantiene cargado el modelo por 30 minutos aunque no se use.

Errores típicos:

* Si el servidor Ollama no está corriendo, no habrá respuesta.

* Puede que falte descargar el modelo (ollama pull qwen2.5:3b).

Paso 4 — Sesión HTTP y función de voz hablar

In [ ]:
# Reutiliza la conexión HTTP
session = requests.Session()

# Reutiliza motor TTS en Windows
_tts_engine = None

def hablar(texto, voz=VOZ):
    global _tts_engine
    texto_limpio = texto.replace("*", "").replace("_", "").replace("#", "")
    if SO == "Darwin":
        os.system(f'say -v "{voz}" "{texto_limpio}"')
    elif SO == "Windows":
        try:
            if _tts_engine is None:
                _tts_engine = pyttsx3.init()
                if voz:
                    for v in _tts_engine.getProperty('voices'):
                        if voz.lower() in v.name.lower():
                            _tts_engine.setProperty('voice', v.id)
                            break
                _tts_engine.setProperty('rate', 160)
                _tts_engine.setProperty('volume', 0.9)
            _tts_engine.say(texto_limpio)
            _tts_engine.runAndWait()
        except Exception as e:
            print(f"Error en TTS: {e}")

¿Qué hace este bloque?

* Crea una sesión HTTP para hacer peticiones al servidor de IA.

* Define la función hablar() para que el personaje pueda leer la respuesta en voz alta.

* Ajusta voz, velocidad y volumen según el sistema operativo.



Paso 5 — Función main: ventana y contenedores base

In [ ]:
def main(page: ft.Page):
    page.title = f"Chat con {PERSONAJE}"
    page.bgcolor = ft.Colors.GREY_100

    mensajes = ft.ListView(
        expand=True, spacing=10, padding=20, auto_scroll=True
    )

¿Qué hace este bloque?

* Crea la ventana principal del chat.

* Muestra un área donde aparecerán los mensajes.

* auto_scroll=True hace que la vista se mueva sola hacia abajo.

Paso 6 — Burbujas de chat (usuario vs personaje)


Dibuja cada mensaje en una “burbuja” con color/alineación distintos para usuario y personaje. Se usa dentro de main().

In [ ]:
def burbuja(texto, es_usuario):
    return ft.Row(
        [
            ft.Text(EMOJI_USUARIO if es_usuario else EMOJI_PERSONAJE, size=24),
            ft.Container(
                content=ft.Text(
                    texto,
                    color=ft.Colors.WHITE if es_usuario else ft.Colors.BLACK,
                    size=15,
                    selectable=True,
                ),
                bgcolor=ft.Colors.BLUE_400 if es_usuario else ft.Colors.GREY_300,
                padding=12,
                border_radius=30,
                shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                margin=ft.margin.only(left=10) if es_usuario else ft.margin.only(right=10),
                alignment=ft.alignment.center_right if es_usuario else ft.alignment.center_left,
                width=350,
            )
        ] if es_usuario else [
            ft.Container(
                content=ft.Text(
                    texto,
                    color=ft.Colors.BLACK,
                    size=15,
                    selectable=True,
                ),
                bgcolor=ft.Colors.GREY_300,
                padding=12,
                border_radius=30,
                shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                margin=ft.margin.only(right=10),
                alignment=ft.alignment.center_left,
                width=350,
            ),
            ft.Text(EMOJI_PERSONAJE, size=24),
        ],
        alignment=ft.MainAxisAlignment.END if es_usuario else ft.MainAxisAlignment.START,
    )


¿Qué hace este bloque?

* Dibuja cada mensaje como burbuja.

* Cambia color/alineación según si habla el usuario o el personaje.

* Muestra el emoji correspondiente a cada lado.

Paso 7 — Entrada de texto y checkbox de voz

In [ ]:
prompt = ft.TextField(
    label="Escribe tu mensaje...",
    expand=True,
    border_radius=20,
    filled=True,
    bgcolor=ft.Colors.WHITE,
    multiline=True,
    min_lines=1,
    max_lines=4,
)

voz_activada = ft.Checkbox(label="🔊 Leer respuestas en voz alta", value=True)


¿Qué hace este bloque?

* TextField para que el alumno escriba su mensaje (hasta 4 líneas).

* Checkbox para activar/desactivar que el personaje lea en voz alta (TTS).

Paso 8 — enviar_click: preguntar al modelo y streaming de respuesta

In [ ]:
def enviar_click(e):
    user_input = prompt.value.strip()
    if not user_input:
        return

    # Muestra mensaje del usuario
    mensajes.controls.append(burbuja(user_input, es_usuario=True))
    page.update()
    prompt.value = ""
    page.update()

    # Construir prompt
    prompt_personaje = (
        f"Responde como si fueras {PERSONAJE}. "
        "Habla con su estilo, conocimientos y personalidad. "
        "Responde en español de manera clara y concisa. "
        f"Pregunta del usuario: {user_input}"
    )

    # Crear contenedor vacío para la respuesta
    respuesta_live = ft.Text("", color=ft.Colors.BLACK, size=15, selectable=True)
    contenedor_bot = ft.Row([
        ft.Container(
            content=respuesta_live,
            bgcolor=ft.Colors.GREY_300,
            padding=12,
            border_radius=30,
            width=350,
        ),
        ft.Text(EMOJI_PERSONAJE, size=24),
    ], alignment=ft.MainAxisAlignment.START)

    mensajes.controls.append(contenedor_bot)
    page.update()

    try:
        resp = session.post(
            OLLAMA_URL,
            json={
                "model": MODEL,
                "prompt": prompt_personaje,
                "stream": True,
                "keep_alive": KEEP_ALIVE,
                "options": OLLAMA_OPTIONS
            },
            stream=True,
            timeout=300,
        )
        resp.raise_for_status()

        texto_final = ""
        # 🔥 Streaming incremental: escribe token por token
        for line in resp.iter_lines():
            if not line:
                continue
            data = json.loads(line)
            if "response" in data:
                chunk = data["response"]
                texto_final += chunk
                respuesta_live.value = texto_final
                page.update()   # refresca pantalla en cada token
            elif "error" in data:
                texto_final = f"Error de Ollama: {data['error']}"
                break

        if not texto_final:
            texto_final = "No se recibió respuesta del modelo."

        respuesta_live.value = texto_final
        page.update()

        # TTS al final (no en streaming)
        if voz_activada.value and VOZ:
            try:
                hablar(texto_final, voz=VOZ)
            except Exception as ex:
                print(f"Error en TTS: {ex}")

    except Exception as ex:
        respuesta_live.value = f"Error de conexión o inesperado: {ex}"
        page.update()


¿Qué hace este bloque?

* Agrega la burbuja del alumno y limpia el input.

* Manda la petición a Ollama y muestra la respuesta en tiempo real.

* Si la casilla está activa y hay voz disponible, lee en voz alta al final.

Paso 9 — Eventos y utilidades (Enter, probar voz, limpiar chat)

In [ ]:
prompt.on_submit = enviar_click

def probar_voz(e):
    if VOZ:
        hablar(f"Hola, soy {PERSONAJE}. Esta es mi voz.", voz=VOZ)

def limpiar_chat(e):
    mensajes.controls.clear()
    page.update()


¿Qué hace este bloque?

* Enter envía el mensaje (on_submit).

* Botón 🎤 Probar voz para escuchar la voz configurada.

* 🧹 Limpiar chat borra el historial visual.

Paso 10 — Header, layout final y arranque

In [ ]:
# --- dentro de main ---
header = ft.Container(
    content=ft.Row([
        ft.Text(EMOJI_PERSONAJE, size=32),
        ft.Text(PERSONAJE, size=22, weight="bold", color=ft.Colors.BLUE_900),
    ], alignment=ft.MainAxisAlignment.START, spacing=15),
    padding=ft.padding.symmetric(vertical=16, horizontal=10),
    bgcolor=ft.Colors.WHITE,
    border_radius=ft.border_radius.only(top_left=20, top_right=20),
    shadow=ft.BoxShadow(blur_radius=12, color=ft.Colors.GREY_300, offset=ft.Offset(0, 2))
)

page.add(
    ft.Container(
        content=ft.Column([
            header,
            mensajes,
            ft.Row([
                voz_activada,
                ft.ElevatedButton("🎤 Probar voz", on_click=probar_voz, bgcolor=ft.Colors.GREEN_400, color=ft.Colors.WHITE),
                ft.TextButton("🧹 Limpiar chat", on_click=limpiar_chat),
            ], alignment=ft.MainAxisAlignment.START, spacing=10),
            ft.Row([
                prompt,
                ft.ElevatedButton("Enviar", on_click=enviar_click, bgcolor=ft.Colors.BLUE_400, color=ft.Colors.WHITE),
            ], vertical_alignment=ft.CrossAxisAlignment.END),
        ], expand=True, spacing=10),
        expand=True,
        padding=0,
        border_radius=0,
        bgcolor=ft.Colors.WHITE,
    )
)

# --- fuera de main ---
ft.app(target=main)


¿Qué hace este bloque?

* Crea la cabecera (emoji + nombre del personaje).

* Organiza la interfaz (historial, controles de voz, botones, input).

* Lanza la app con ft.app(target=main).

Codigó Completo

In [ ]:
import flet as ft
import requests
import json
import pyttsx3
import platform
import os

# ======== CONFIG ========
SO = platform.system()

if SO == "Darwin":  # Mac
    PERSONAJE = "Albert Einstein"
    EMOJI_PERSONAJE = "🧑‍🔬"
    VOZ = "Juan"
elif SO == "Windows":
    PERSONAJE = "Marie Curie"
    EMOJI_PERSONAJE = "👩‍🔬"
    VOZ = "Sabina"  # o "Zira"
else:
    PERSONAJE = "Personaje"
    EMOJI_PERSONAJE = "🧑‍🔬"
    VOZ = None

EMOJI_USUARIO = "🧑‍💻"
OLLAMA_URL = "http://localhost:11434/api/generate"

# Modelo ligero para velocidad
MODEL = "qwen2.5:3b"  # puedes probar "gemma3:latest" o "mistral:7b"

# Opciones ajustadas para rapidez
OLLAMA_OPTIONS = {
    "num_ctx": 4096,       # tokens de contexto
    "num_predict": 512,    # tokens máximos de salida
    "temperature": 0.7,
    "top_p": 0.9,
    "repeat_penalty": 1.1,
}
KEEP_ALIVE = "30m"

# Reutiliza la conexión HTTP
session = requests.Session()

# Reutiliza motor TTS en Windows
_tts_engine = None
def hablar(texto, voz=VOZ):
    global _tts_engine
    texto_limpio = texto.replace("*", "").replace("_", "").replace("#", "")
    if SO == "Darwin":
        os.system(f'say -v "{voz}" "{texto_limpio}"')
    elif SO == "Windows":
        try:
            if _tts_engine is None:
                _tts_engine = pyttsx3.init()
                if voz:
                    for v in _tts_engine.getProperty('voices'):
                        if voz.lower() in v.name.lower():
                            _tts_engine.setProperty('voice', v.id)
                            break
                _tts_engine.setProperty('rate', 160)
                _tts_engine.setProperty('volume', 0.9)
            _tts_engine.say(texto_limpio)
            _tts_engine.runAndWait()
        except Exception as e:
            print(f"Error en TTS: {e}")

def main(page: ft.Page):
    page.title = f"Chat con {PERSONAJE}"
    page.bgcolor = ft.Colors.GREY_100

    mensajes = ft.ListView(expand=True, spacing=10, padding=20, auto_scroll=True)

    def burbuja(texto, es_usuario):
        return ft.Row(
            [
                ft.Text(EMOJI_USUARIO if es_usuario else EMOJI_PERSONAJE, size=24),
                ft.Container(
                    content=ft.Text(
                        texto,
                        color=ft.Colors.WHITE if es_usuario else ft.Colors.BLACK,
                        size=15,
                        selectable=True,
                    ),
                    bgcolor=ft.Colors.BLUE_400 if es_usuario else ft.Colors.GREY_300,
                    padding=12,
                    border_radius=30,
                    shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                    margin=ft.margin.only(left=10) if es_usuario else ft.margin.only(right=10),
                    alignment=ft.alignment.center_right if es_usuario else ft.alignment.center_left,
                    width=350,
                )
            ] if es_usuario else [
                ft.Container(
                    content=ft.Text(
                        texto,
                        color=ft.Colors.BLACK,
                        size=15,
                        selectable=True,
                    ),
                    bgcolor=ft.Colors.GREY_300,
                    padding=12,
                    border_radius=30,
                    shadow=ft.BoxShadow(blur_radius=8, color=ft.Colors.GREY_400, offset=ft.Offset(2, 2)),
                    margin=ft.margin.only(right=10),
                    alignment=ft.alignment.center_left,
                    width=350,
                ),
                ft.Text(EMOJI_PERSONAJE, size=24),
            ],
            alignment=ft.MainAxisAlignment.END if es_usuario else ft.MainAxisAlignment.START,
        )

    prompt = ft.TextField(
        label="Escribe tu mensaje...",
        expand=True,
        border_radius=20,
        filled=True,
        bgcolor=ft.Colors.WHITE,
        multiline=True,
        min_lines=1,
        max_lines=4,
    )

    voz_activada = ft.Checkbox(label="🔊 Leer respuestas en voz alta", value=True)

    def enviar_click(e):
        user_input = prompt.value.strip()
        if not user_input:
            return

        # Muestra mensaje del usuario
        mensajes.controls.append(burbuja(user_input, es_usuario=True))
        page.update()
        prompt.value = ""
        page.update()

        # Construir prompt
        prompt_personaje = (
            f"Responde como si fueras {PERSONAJE}. "
            "Habla con su estilo, conocimientos y personalidad. "
            "Responde en español de manera clara y concisa. "
            f"Pregunta del usuario: {user_input}"
        )

        # Crear contenedor vacío para la respuesta
        respuesta_live = ft.Text("", color=ft.Colors.BLACK, size=15, selectable=True)
        contenedor_bot = ft.Row([
            ft.Container(
                content=respuesta_live,
                bgcolor=ft.Colors.GREY_300,
                padding=12,
                border_radius=30,
                width=350,
            ),
            ft.Text(EMOJI_PERSONAJE, size=24),
        ], alignment=ft.MainAxisAlignment.START)

        mensajes.controls.append(contenedor_bot)
        page.update()

        try:
            resp = session.post(
                OLLAMA_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt_personaje,
                    "stream": True,
                    "keep_alive": KEEP_ALIVE,
                    "options": OLLAMA_OPTIONS
                },
                stream=True,
                timeout=300,
            )
            resp.raise_for_status()

            texto_final = ""
            # 🔥 Streaming incremental: escribe token por token
            for line in resp.iter_lines():
                if not line:
                    continue
                data = json.loads(line)
                if "response" in data:
                    chunk = data["response"]
                    texto_final += chunk
                    respuesta_live.value = texto_final
                    page.update()   # refresca pantalla en cada token
                elif "error" in data:
                    texto_final = f"Error de Ollama: {data['error']}"
                    break

            if not texto_final:
                texto_final = "No se recibió respuesta del modelo."

            respuesta_live.value = texto_final
            page.update()

            # TTS al final (no en streaming)
            if voz_activada.value and VOZ:
                try:
                    hablar(texto_final, voz=VOZ)
                except Exception as ex:
                    print(f"Error en TTS: {ex}")

        except Exception as ex:
            respuesta_live.value = f"Error de conexión o inesperado: {ex}"
            page.update()

    prompt.on_submit = enviar_click

    def probar_voz(e):
        if VOZ:
            hablar(f"Hola, soy {PERSONAJE}. Esta es mi voz.", voz=VOZ)

    def limpiar_chat(e):
        mensajes.controls.clear()
        page.update()

    header = ft.Container(
        content=ft.Row([
            ft.Text(EMOJI_PERSONAJE, size=32),
            ft.Text(PERSONAJE, size=22, weight="bold", color=ft.Colors.BLUE_900),
        ], alignment=ft.MainAxisAlignment.START, spacing=15),
        padding=ft.padding.symmetric(vertical=16, horizontal=10),
        bgcolor=ft.Colors.WHITE,
        border_radius=ft.border_radius.only(top_left=20, top_right=20),
        shadow=ft.BoxShadow(blur_radius=12, color=ft.Colors.GREY_300, offset=ft.Offset(0, 2))
    )

    page.add(
        ft.Container(
            content=ft.Column([
                header,
                mensajes,
                ft.Row([
                    voz_activada,
                    ft.ElevatedButton("🎤 Probar voz", on_click=probar_voz, bgcolor=ft.Colors.GREEN_400, color=ft.Colors.WHITE),
                    ft.TextButton("🧹 Limpiar chat", on_click=limpiar_chat),
                ], alignment=ft.MainAxisAlignment.START, spacing=10),
                ft.Row([
                    prompt,
                    ft.ElevatedButton("Enviar", on_click=enviar_click, bgcolor=ft.Colors.BLUE_400, color=ft.Colors.WHITE),
                ], vertical_alignment=ft.CrossAxisAlignment.END),
            ], expand=True, spacing=10),
            expand=True,
            padding=0,
            border_radius=0,
            bgcolor=ft.Colors.WHITE,
        )
    )

ft.app(target=main)


#Responde estas preguntas en tu google sites.

1. ¿Qué versiones de Python se recomiendan para este proyecto?

2. ¿Por qué se necesita conexión a internet al menos la primera vez?

3. ¿Qué comando crea la carpeta base del proyecto con Flet?

4. Escribe el comando para activar tu venv en tu sistema (macOS o Windows).

5. ¿Desde qué URL oficial se descarga Ollama?

6. ¿Qué comando descarga el modelo qwen2.5:3b?

7. ¿Qué modelo de IA usa el proyecto por defecto?

8. ¿Para qué sirve la variable KEEP_ALIVE?

9. ¿Qué librería se usa para la interfaz gráfica?

10. ¿Qué significa stream=True al llamar al endpoint de Ollama?

11. ¿Qué controla la casilla “🔊 Leer respuestas en voz alta”?

12. ¿Qué ocurre si el TextField está vacío cuando presionas Enviar?

13. ¿Qué hace la línea prompt.on_submit = enviar_click?

14. ¿Con qué instrucción se lanza la app de Flet?

15. Si qwen2.5:3b no aparece en ollama list, ¿qué comando debes ejecutar para solucionarlo?